In [3]:
import pandas as pd
#from sklearn import tree
import graphviz
#from sklearn.model_selection import train_test_split
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics

df = pd.read_csv("file:/home/hduser/pythonwork/dmhw1208/data/character-deaths.csv")#讀取資料
df=df.fillna(0)#把空值以0替代
df.loc[df['Death Year'] > 0, 'Death Year'] = 1.0
df=df.drop('Book of Death',axis = 1)
df=df.drop('Death Chapter',axis = 1)
df = pd.get_dummies(df, columns=['Allegiances'])#將Allegiances底下的家族轉成dummy的特徵
df = df.drop('Name',axis = 1)

#------------


x = df.drop('Death Year', axis=1)
y = df['Death Year']


from sklearn.datasets import dump_svmlight_file
dump_svmlight_file(x, y, 'svm-output.libsvm')  # where is your y?
from sklearn.datasets import load_svmlight_file
#data = load_svmlight_file("svm-output.libsvm")
#labelIndexer = StringIndexer(inputCol="Death Year", outputCol="Death Year").fit(df)


#X = df.drop('Death Year', axis=1)
#y = df['Death Year']

#X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.75,test_size=0.25)#前75%是訓練集、後25%當測試集

#model=DecisionTree.trainClassifier(X_train,2,y_train,"gini",10,2)

#model = tree.DecisionTreeClassifier()
#model = model.fit(X_train, y_train)

#dot_data = tree.export_graphviz(model, out_file=None,max_depth=6)#限制樹的深度，以免結果無法顯示
#graph = graphviz.Source(dot_data)
#graph.render("df", view=True)

In [8]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.mllib.evaluation import MultilabelMetrics

data  = MLUtils.loadLibSVMFile(sc,"svm-output.libsvm")
(trainingData, testData) = data.randomSplit([0.75, 0.25])
model = DecisionTree.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},impurity='gini', maxDepth=5, maxBins=32)

predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)

#[0]:正確 [1]:預測

#Accuracy = labelsAndPredictions.filter(lambda lp: lp[0] == lp[1]).count() / float(testData.count())
#Recall = labelsAndPredictions.filter(lambda lp: lp[0] == lp[1]==1).count() / float(labelsAndPredictions.filter(lambda lp: lp[0]==1).count())
#Precision = labelsAndPredictions.filter(lambda lp: lp[0] == lp[1]==1).count() / float(labelsAndPredictions.filter(lambda lp: lp[1]==1).count())
#print("Summary Stats:")
#print('accuracy = ' + str(Accuracy))
#print('Recall = ' + str(Recall))
#print('Precision = ' + str(Precision))


#zz=labelsAndPredictions.take(20000)
#scoreAndLabels = sc.parallelize(zz)

#算出來怪怪ㄉ 修好了！-------------------------
metrics = MulticlassMetrics(labelsAndPredictions)
precision = metrics.precision(label=1)
recall = metrics.recall(label=1)
Accuracy = metrics.accuracy
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("Accuracy = %s" % Accuracy)
#算出來怪怪ㄉ-------------------------







#print('Learned classification tree model:')
#print(model.toDebugString())


#model.save(sc, "myDecisionTreeClassificationModel")
#sameModel = DecisionTreeModel.load(sc, "myDecisionTreeClassificationModel")


Precision = 0.626666666667
Recall = 0.61038961039
Accuracy = 0.725118483412
